
* Overview of GitHub Repository Variables
* Create GitHub Repository Secrets
* Run Commands on VM using SSH
* Validate GitHub Action and VM Integration
* Copying Files to the Virtual Machine
* Deploy Demo App on to the VM using GitHub Action
* Exercise and Solution - Deploy Applications using GitHub Actions

* Overview of GitHub Repository Variables

Create Variable by name `FOO` with value `bar`.

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Hello World
      run: |
        echo ${{ vars.FOO }}
        echo `hostname`
        echo "Hello World"
```

* Create GitHub Repository Secrets

Here are the details to create GitHub Repository Secrets. Secrets can be created by going to Settings of a specific repository.
|Secret Name|Description|
|---|---|
|APP_HOST|IP Address or DNS Alias of VM|
|APP_SSH_USER|Username to connect to VM|
|APP_SSH_SECRET_KEY|Private key for password less login|

As the VM is created using GCP, the content of `google_compute_engine` is used for `APP_SSH_SECRET_KEY`.

* Run Commands on VM using SSH
  * We can run command using SSH - (syntax: **ssh -i path_to_private_key username@hostname "command"**)
  * Example - `ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "ls -ltr"`
  * We can run any valid linux command using the above approach.

* Validate GitHub Action and VM Integration

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Validate GitHub Action and VM Integration
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          echo `hostname`
        '
```

* Copying Files to the Virtual Machine
  * scp - secure copy
  * rsync - remote sync

scp syntax - `scp -i path_to_private_key path_to_local_files username@ip_or_dns_alias_of_target_vm:path_on_target_vm`

eg: `scp -i ~/.ssh/google_compute_engine * itversity@35.238.90.14:~/copydemo`

Here are the commands used for scp demo for your reference:
```shell
mkdir copydemo # On Windows you can also use VS Code IDE to perform these steps
cp app.py copydemo
cd copydemo
touch testdir
cp app.py testdir

ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "ls -ltr ~/copydemo"
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "mkdir ~/copydemo"
scp -i ~/.ssh/google_compute_engine -r * itversity@35.238.90.14:~/copydemo
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "ls -ltr ~/copydemo"
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "find ~/copydemo"
```

rsync syntax: `rsync -avzh path_on_source username@ext_ip_or_dns:path_on_target_host --rsh "ssh -i path_to_private_key_file"`

eg: `rsync -avzh . itversity@35.238.90.14:~/scpdemo --rsh "ssh -i ~/.ssh/google_compute_engine"`

Here are the commands used for rsync demo for your reference:

```shell
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "rm -rf ~/copydemo"
rsync -avzh * itversity@35.238.90.14:~/copydemo --rsh "ssh -i ~/.ssh/google_compute_engine"
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "find ~/copydemo"
```

Before moving on, let us make sure `copydemo` folder is deleted from both local as well as target.

```shell
rm -rf copydemo # You can also use VS Code IDE to delete on Windows/Mac
ssh -i ~/.ssh/google_compute_engine itversity@35.238.90.14 "rm -rf ~/copydemo"
```

* Deploy Demo App on to the VM using GitHub Action

Here are the steps to deploy demo app on to the VM using GitHub Action.
1. Delete the folder related to the repo from the target VM.
2. Create a local branch to make changes.
3. Update GitHub Action workflow yaml file with required instructions.

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Set Environment Variables
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p ~/cicd-demo
        '
    - name: Deploy Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        scp -i /tmp/keys/private_key * ${APP_SSH_USER}@${APP_HOST}:~/cicd-demo
```

4. Push the newly created branch to origin (remote repository)
5. Create PR and merge changes
6. Review Action to see if it is run successfully or not.
7. Connect to VM and validate to see if the folder is successfully copied or not.

* Exercise - Deploy Applications using GitHub Actions

Deploy sales-app on to the target VM.
1. Delete the folder related to the repo from the target VM.
2. Create a local branch to make changes.
3. Update GitHub Action workflow yaml file with required instructions. Use cicd-demo as reference.
4. Push the newly created branch to origin (remote repository)
5. Create PR and merge changes
6. Review Action to see if it is run successfully or not.
7. Connect to VM and validate to see if the folder is successfully copied or not.

* Solution - Deploy Applications using GitHub Actions

```yml
name: Sales App

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Set Environment Variables
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p ~/sales-app
        '
    - name: Deploy Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        scp -i /tmp/keys/private_key * ${APP_SSH_USER}@${APP_HOST}:~/sales-app
```